Dans ce billet, nous allons [comme il y a quelques semaines](http://flothesof.github.io/podcast-rendez-vous-avec-X-fr.html), construire un flux RSS pour mon émission radio préférée : *Sur les épaules de Darwin*, animée par l'extraordinaire Jean Claude Ameisen.

Afin de faire ceci, nous allons partir du [travail de Clément Grimal](http://clementgrimal.fr/darwin/), qui maintient une liste à jour des émissions avec des liens de téléchargement. 

Une question, avant de commencer. Pourquoi faire ce travail, alors que le site de Clément permet déjà d'écouter à volonté ces émissions ? Parce que son site ne permet pas de faire une recherche plein texte sur le contenu des émissions et également parce que j'aime écouter les émissions dans un logiciel de podcast afin de garder un historique de mes écoutes (j'utilise [Podcast Addict](https://play.google.com/store/apps/details?id=com.bambuna.podcastaddict&hl=en)).


**TL;DR**
Le lien vers le flux RSS généré à partir du présent notebook est ici : *https://raw.githubusercontent.com/flothesof/posts/master/files/podcast_Sur_les_epaules_de_Darwin.xml*

Date de dernière mise à jour : *avril 2020*.

# Récupération des liens vers les émissions

Comme je le disais, notre point de départ est le site de Clément Grimal. Nous analysons sa source avec `beautifulsoup4` pour en extraire les éléments qui nous intéressent :


- titre de l'émission
- date de diffusion
- lien de téléchargement
- lien vers la page du site France Inter

In [1]:
import requests
from bs4 import BeautifulSoup


base_url = "http://clementgrimal.fr/darwin/"

r = requests.get(base_url)
r.encoding = 'utf-8'

soup = BeautifulSoup(r.text, 'html.parser')

L'objet `soup` nous permet d'extraire les différents éléments de la page web, de manière structurée. On obtient la liste de toutes les émissions de cette manière :

In [2]:
all_shows = ([item for item in (li.find('table') for li in soup.findAll('li')) if item])

len(all_shows)

516

A partir du tag `li` (item de liste en HTML), on peut obtenir le lien vers le fichier mp3 ainsi que le site France Inter, de même que le titre et la date de diffusion.

Par exemple avec le dernier épisode : 

In [3]:
show = all_shows[-1]

show.find('a', class_='download-link').attrs['href'].strip()

'./files/2020-04-11 - aux-origines-du-chocolat-8.mp3'

In [4]:
show.find('span').find('a').attrs['href']

'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-11-avril-2020'

In [5]:
show.find('span').find('a').text

'Aux Origines du Chocolat (8)'

In [6]:
str(show.find('span').find('a').next_sibling)[14:]

'11 Avril 2020'

Ecrivons maintenant une boucle pour extraire ces informations pour l'intégralité des épisodes :

In [19]:
show_data = []
for show in all_shows:
    download_link = show.find('a', class_='download-link')
    description_link = show.find('span').find('a').attrs['href']
    title = show.find('span').find('a').text
    date = str(show.find('span').find('a').next_sibling)[14:]
    if download_link:
        if download_link.attrs['href'].strip().startswith('.'):
            download_link = "http://clementgrimal.fr/darwin" + download_link.attrs['href'].strip()[1:]
        else:
            download_link = download_link.attrs['href'].strip()
        show_data.append([download_link, description_link, title, date])
    else: 
        print(f"lien de téléchargement indisponible pour l'émission {show.find('a').attrs['href']}")

lien de téléchargement indisponible pour l'émission https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-12-janvier-2013
lien de téléchargement indisponible pour l'émission https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-28-mars-2015
lien de téléchargement indisponible pour l'émission https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-04-avril-2015
lien de téléchargement indisponible pour l'émission https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-11-avril-2015
lien de téléchargement indisponible pour l'émission https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-02-juin-2019
lien de téléchargement indisponible pour l'émission https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-09-juin-2019
lien de téléchargement indisponible pour l'émission https://www.francei

Construisons une dataframe `pandas` avec ces données :

In [20]:
import pandas as pd

df = pd.DataFrame(show_data, columns=['lien mp3', 'lien description', 'titre', 'date'])
df.head(10)

,lien mp3,lien description,titre,date
0,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,La théorie de l'évolution de Charles Darwin,4 Septembre 2010
1,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Le propre de l'homme?,11 Septembre 2010
2,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos mémoires,18 Septembre 2010
3,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos émotions,25 Septembre 2010
4,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,"Longévité, jeunesse et vieillissement",2 Octobre 2010
5,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,MORT CELLULAIRE ET SCULPTURE DU VIVANT,9 Octobre 2010
6,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,NAISSANCES,16 Octobre 2010
7,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Biodiversité,23 Octobre 2010
8,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,mort cellulaire et sculpture du vivant (2),30 Octobre 2010
9,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Un voyage avec Oliver Sacks ( 1 ),6 Novembre 2010


In [21]:
df.describe()

,lien mp3,lien description,titre,date
count,497,497,497,497
unique,497,497,394,497
top,http://clementgrimal.fr/darwin/files/2011-04-2...,https://www.franceinter.fr/emissions/sur-les-e...,Le souci de l'autre,7 Mai 2016
freq,1,1,3,1


Nous voilà maintenant en possession des liens vers 497 épisodes publiés avec lien de téléchargement sur le site de Clément Grimal. On remarque que certaines d'entre elles sont des rediffusions (394 titres uniques sur 497). 

Téléchargeons maintenant les descriptions des émissions, afin de construire un fil RSS avec celles-ci.

# Téléchargement des descriptions à partir du site de France Inter 

Nous avons les liens vers la page de chaque émission : si on lit celle-ci, on peut en extraire le contenu que nous cherchons, c'est-à-dire la description de l'épisode.

Essayons de faire cela avec le dernier épisode en date.

In [47]:
link = df['lien description'].iloc[-1]
link

'https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-11-avril-2020'

In [59]:
soup = BeautifulSoup(requests.get(link).text, 'html.parser')

In [60]:
article_tag = soup.find('article')
article_tag

<article class="content-body">
<p class="chapo">
            Merci à vous, (un million six cent quatre vingt mille) auditeurs fidèles 
Portez-vous bien ainsi que tous les vôtres
        </p>
<figure class="rich-visual"><picture><source class="dejavu" data-dejavu-srcset="https://cdn.radiofrance.fr/s3/cruiser-production/2020/04/997ff263-9655-4a2d-9504-3b5ca104b734/640_gettyimages-960923834.webp" srcset="" type="image/webp"/><img alt="le chocolat" class="dejavu" data-dejavu-src="https://cdn.radiofrance.fr/s3/cruiser-production/2020/04/997ff263-9655-4a2d-9504-3b5ca104b734/640_gettyimages-960923834.jpg" height="391" src="data:image/jpeg;base64, /9j/2wBDACgcHiMeGSgjISMtKygwPGRBPDc3PHtYXUlkkYCZlo+AjIqgtObDoKrarYqMyP/L2u71////m8H////6/+b9//j/2wBDASstLTw1PHZBQXb4pYyl+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj/wAARCAAZACoDASIAAhEBAxEB/8QAFwAAAwEAAAAAAAAAAAAAAAAAAQIEA//EACIQAAICAQMFAQEAAAAAAAAAAAECAAMREiExBBMyQWEiYv/EABYBAQEBAAAAAAAAAAAAAAAAAAECAP/EABYRAQEBAAAAAAAAAAAAAAAA

Il se trouve que des balises script sont contenues dans les épisodes (elles affichent de la publicité sur le site de France Inter).

Si on reformate les tags HTML, on obtient le résultat suivant :

In [114]:
soup.find_all('section')[0].text

"AccueilÉmissions\n                                                                    Sur les épaules de Darwin\n                                                            Aux Origines du Chocolat (8)                            Sur les épaules de Darwin\n                        samedi 11 avril 2020\n                                                            par\n                                Jean Claude AmeisenAux Origines du Chocolat (8)\n                                                        53 minutes\n                        réécouters'abonnerPodcastRSSréagirContact"

In [74]:
def clean(soup, tags_to_clean=['script']):
    """Nettoie le HTML de l'émission en enlevant les balises inutiles."""
    for tag in tags_to_clean:
        for item in soup.find_all(tag):
            item.decompose()
    return soup.article

clean(soup)

<article class="content-body">
<p class="chapo">
            Merci à vous, (un million six cent quatre vingt mille) auditeurs fidèles 
Portez-vous bien ainsi que tous les vôtres
        </p>
<figure class="rich-visual"><picture><source class="dejavu" data-dejavu-srcset="https://cdn.radiofrance.fr/s3/cruiser-production/2020/04/997ff263-9655-4a2d-9504-3b5ca104b734/640_gettyimages-960923834.webp" srcset="" type="image/webp"/><img alt="le chocolat" class="dejavu" data-dejavu-src="https://cdn.radiofrance.fr/s3/cruiser-production/2020/04/997ff263-9655-4a2d-9504-3b5ca104b734/640_gettyimages-960923834.jpg" height="391" src="data:image/jpeg;base64, /9j/2wBDACgcHiMeGSgjISMtKygwPGRBPDc3PHtYXUlkkYCZlo+AjIqgtObDoKrarYqMyP/L2u71////m8H////6/+b9//j/2wBDASstLTw1PHZBQXb4pYyl+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj/wAARCAAZACoDASIAAhEBAxEB/8QAFwAAAwEAAAAAAAAAAAAAAAAAAQIEA//EACIQAAICAQMFAQEAAAAAAAAAAAECAAMREiExBBMyQWEiYv/EABYBAQEBAAAAAAAAAAAAAAAAAAECAP/EABYRAQEBAAAAAAAAAAAAAAAA

On peut afficher un aperçu de notre description :

In [77]:
from IPython.display import HTML

HTML(str(clean(soup)))

Ceci étant fait, on peut écrire une boucle pour extraire toutes les descriptions d'épisode :

In [78]:
from functools import lru_cache

@lru_cache(maxsize=None)
def link_getter(description_link):
    """Récupère la page de l'émission en question, stocke le résultat pour un accès ultérieur plus rapide."""
    return requests.get(description_link)

In [101]:
import tqdm

descriptions = []
for description_link in tqdm.tqdm(df['lien description']):
    soup = BeautifulSoup(link_getter(description_link).text, 'html.parser')
    descriptions.append(clean(soup))

100%|██████████| 497/497 [00:50<00:00,  9.87it/s]


On peut compléter notre `dataframe` avec les descriptions obtenues :

In [102]:
df['description'] = descriptions

df['description_text'] = [d.text for d in descriptions]

Afin de vérifier que tout va bien, nous allons faire un pré-affichage rapide avec `ipywidgets`.

In [104]:
from ipywidgets import interact

@interact
def preview_html(n=(0, df.shape[0] - 1)):
    return HTML(f"<a href='{df['lien description'].iloc[n]}' target='_blank'>lien</a>" + str(df.description.iloc[n]))

interactive(children=(IntSlider(value=248, description='n', max=496), Output()), _dom_classes=('widget-interac…

# Rajouter d'autres sections 

Comme on peut le voir sur cet épisode [https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin/sur-les-epaules-de-darwin-16-mai-2015], parfois les références sont cachées dans un tag `section`, avec le nom de l'équipe de production. Comptons le nombre de tag `section` :

In [135]:
sections = []
for description_link in tqdm.tqdm(df['lien description']):
    soup = BeautifulSoup(link_getter(description_link).text, 'html.parser')
    sections.append(len(soup.find('div', class_='main-content').find_all('section')))

100%|██████████| 497/497 [00:40<00:00, 12.24it/s]


Et regardons le comptage :

In [138]:
pd.Series(sections).value_counts()

2    295
1    189
3     10
0      3
dtype: int64

On peut donc proposer un deuxième algorithme : on va chercher les sections supplémentaires en-dessous de `main-content` et on les nettoie elles-aussi.

In [182]:
def clean2(soup, tags_to_clean=['script']):
    """Nettoie le HTML de l'émission en enlevant les balises inutiles."""
    for tag in tags_to_clean:
        for item in soup.find_all(tag):
            item.decompose()
    return str(soup.find('div', class_='main-content').article) + "\n".join([str(item) for item in soup.find('div', class_='main-content').find_all('section', class_='content-body-more-details')])


In [185]:
HTML(clean2(soup))

On met à jour les descriptions.

In [186]:
descriptions = []
for description_link in tqdm.tqdm(df['lien description']):
    soup = BeautifulSoup(link_getter(description_link).text, 'html.parser')
    descriptions.append(clean2(soup))

100%|██████████| 497/497 [00:48<00:00, 10.21it/s]


In [187]:
df['description'] = descriptions

df['description_text'] = [str(d) for d in descriptions]

Et on vérifie qu'on a le résultat attendu.

In [188]:
@interact
def preview_html(n=(0, df.shape[0] - 1)):
    return HTML(f"<a href='{df['lien description'].iloc[n]}' target='_blank'>lien</a>" + str(df.description.iloc[n]))

interactive(children=(IntSlider(value=248, description='n', max=496), Output()), _dom_classes=('widget-interac…

Un dernier test :

In [191]:
soup = BeautifulSoup(link_getter(df['lien description'].iloc[230]).text, 'html.parser')

HTML(clean2(soup))

Il nous manque encore la récupération des tailles de fichier mp3, qui sera inscrite dans le flux RSS :

# Taille des fichiers mp3 

In [115]:
byte_lengths = []
for mp3_link in tqdm.tqdm(df['lien mp3']):
    r = requests.head(mp3_link)
    byte_lengths.append(r.headers['content-length'])

100%|██████████| 497/497 [00:32<00:00, 15.20it/s]


In [116]:
df['byte_length'] = byte_lengths

df.head()

,lien mp3,lien description,titre,date,description,description_text,byte_length
0,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,La théorie de l'évolution de Charles Darwin,4 Septembre 2010,"[\n, [[Bibliographie:]], \n, [[L'origine des e...",\nBibliographie:\nL'origine des espèces de Cha...,51120256
1,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Le propre de l'homme?,11 Septembre 2010,"[\n, [[programmation musicale]], \n, [STAFF BE...",\nprogrammation musicale\nSTAFF BENDA BILILI\n...,52287616
2,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos mémoires,18 Septembre 2010,"[\n, [[programmation musicale]], \n, [Iggy POP...",\nprogrammation musicale\nIggy POP\nKing of th...,51320960
3,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos émotions,25 Septembre 2010,"[\n, [[programmation musicale]], \n, [[EMILY L...",\nprogrammation musicale\nEMILY LOISEAU\nL'aut...,54608000
4,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,"Longévité, jeunesse et vieillissement",2 Octobre 2010,"[\n, [[programmation musicale]], \n, [[Bonga]]...",\nprogrammation musicale\nBonga\nMona ki gni x...,52762752


# Ecriture du fichier RSS 

Maintenant que nous avons collecté toutes les émissions, nous pouvons écrire un ficher RSS, comme nous l'avions déjà fait [dans le cas de l'émission Rendez vous avec X](http://flothesof.github.io/podcast-rendez-vous-avec-X-fr.html).

In [201]:
import dateparser
import xml.etree.cElementTree as ET

rss = ET.Element("rss", version="2.0")
channel = ET.SubElement(rss, "channel")
title = ET.SubElement(channel, "title")
title.text = 'Podcast Sur les épaules de Darwin'
description = ET.SubElement(channel, "description")
description.text = "Podcast inofficiel de l'émission Sur les épaules de Darwin, tiré du site http://clementgrimal.fr/darwin/ et https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin"
for index, row in df.iterrows():
    item = ET.SubElement(channel, "item")
    item_title = ET.SubElement(item, "title")
    item_title.text = row['titre']
    item_description = ET.SubElement(item, "description")
    item_description.text = f"<![CDATA[{row['description'].strip()}"
    item_pubdate = ET.SubElement(item, "pubDate")
    item_pubdate.text = dateparser.parse(row.date).date().strftime('%a, %d %b %Y 11:00:00')
    item_enclosure = ET.SubElement(item, "enclosure", url='{}'.format(row['lien mp3']),
                                   length=row.byte_length,
                                  type="audio/mpeg")
tree = ET.ElementTree(rss)
tree.write("files/podcast_Sur_les_epaules_de_Darwin.xml", encoding='utf-8')

Le lien vers le fichier généré est ici : **https://raw.githubusercontent.com/flothesof/posts/master/files/podcast_Sur_les_epaules_de_Darwin.xml**.

*Ce billet a été écrit à l'aide d'un notebook Jupyter. Son contenu est sous licence BSD. Une vue statique de ce notebook peut être consultée et téléchargée ici : [20170908_PodcastEpaulesDeDarwin.ipynb](http://nbviewer.ipython.org/urls/raw.github.com/flothesof/posts/master/20170908_PodcastEpaulesDeDarwin.ipynb).*